<a href="https://colab.research.google.com/github/omkar806/on-device-experiments/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q langchain openai langchain_openai

In [12]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from datetime import datetime
from langchain.callbacks import get_openai_callback
import os

In [8]:
#getting today's date
today_date = datetime.today().date()
print(today_date)

2024-05-05


In [10]:
#Defining a pydantic model for getting the exact strcutured json from the input query as we want
class Candidate(BaseModel):
    entity: str = Field(description="The company name mentioned in the query (e.g., Flipkart, Amazon).")
    parameter: str = Field(description="The performance metric mentioned in the query (e.g., GMV, revenue, profit)")
    startDate: str = Field(description=f"The start date of the time period for which the metric is requested. By default If the user query does not explicitly mention the start date.Assume start date to be : Today's date(is {today_date}) minus one year.")
    endDate: str = Field(description=f"The end date of the time period for which the metric is requested.By default If the user query does not explicitly mention the start date. Assume the end date to be : Today's date(is {today_date}).")

In [13]:
os.environ["OPENAI_API_KEY"] = "Enter Your OPENAI_API_KEY"
model_name = "gpt-3.5-turbo-instruct"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=800)

In [19]:
while True:
  user_query = input("User: ")
  parser = PydanticOutputParser(pydantic_object=Candidate)
  prompt = PromptTemplate(
    template="Answer the user query.\n {format_instructions}\n {query} \n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
  )
  input = prompt.format_prompt(query=user_query)
  with get_openai_callback() as cb:
      result = model(input.to_string())
      #print(result)
      print(parser.parse(result))
      #print(result)
      #print(cb)

  # print(output)
  # parser.parse(output)

TypeError: 'StringPromptValue' object is not callable

In [ ]:
class_object= parser.parse(result)
dict_object=class_object.__dict__
print(type(dict_object))
print(dict_object)